<a href="https://colab.research.google.com/github/filipecalegario/criacomp/blob/main/2024_1_CRIACOMP_Experimentando_com_GRADIO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Usando Gradio para criar interfaces simples

Confira a biblioteca no [GitHub](https://github.com/gradio-app/gradio-UI) e veja a página de [primeiros passos](https://gradio.app/getting_started.html) para mais demonstrações.

### Definindo uma função

In [ ]:
def greet(name):
  return "Hello " + name + "!"

greet("World")

### Instalando o gradio



In [ ]:
!pip install -q gradio

### Criando uma interface simples para a função

In [ ]:
import gradio

gradio.Interface(greet, "text", "text").launch(share=True)

### Interface com múltiplos parâmetros de entrada

In [ ]:
import gradio as gr

def greet(name, intensity):
    return "Hello " * intensity + name + "!"

demo = gr.Interface(
    fn=greet,
    inputs=["text", "slider"],
    outputs=["text"],
)

demo.launch()

### Parâmetros avançados de interface

In [ ]:
import gradio as gr

def yes_man(message, history):
    if message.endswith("?"):
        return "Yes"
    else:
        return "Ask me anything!"

gr.ChatInterface(
    yes_man,
    chatbot=gr.Chatbot(height=300),
    textbox=gr.Textbox(placeholder="Ask me a yes or no question", container=False, scale=7),
    title="Yes Man",
    description="Ask Yes Man any question",
    theme="soft",
    examples=["Hello", "Am I cool?", "Are tomatoes vegetables?"],
    cache_examples=True,
    retry_btn=None,
    undo_btn="Delete Previous",
    clear_btn="Clear",
).launch()

## Interfaces de Chat

In [ ]:
!pip install -q langchain openai langchain_community gradio

### Usando a biblioteca da OpenAI em Python

In [ ]:
from openai import OpenAI
import gradio as gr
from google.colab import userdata

api_key = userdata.get('OPENAI_KEY')
client = OpenAI(api_key=api_key)

def predict(message, history):
    history_openai_format = []
    for human, assistant in history:
        history_openai_format.append({"role": "user", "content": human })
        history_openai_format.append({"role": "assistant", "content":assistant})
    history_openai_format.append({"role": "user", "content": message})

    response = client.chat.completions.create(model='gpt-4o',
      messages= history_openai_format,
      temperature=1.0,
      stream=True)

    partial_message = ""
    for chunk in response:
        if chunk.choices[0].delta.content is not None:
              partial_message = partial_message + chunk.choices[0].delta.content
              yield partial_message

gr.ChatInterface(predict).launch()

### Usando LangChain com OpenAI

In [ ]:
from google.colab import userdata
from langchain.chat_models import ChatOpenAI
from langchain.schema import AIMessage, HumanMessage
import openai
import gradio as gr
import os

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_KEY')

llm = ChatOpenAI(temperature=1.0, model='gpt-4o')

def predict(message, history):
    history_langchain_format = []
    for human, ai in history:
        history_langchain_format.append(HumanMessage(content=human))
        history_langchain_format.append(AIMessage(content=ai))
    history_langchain_format.append(HumanMessage(content=message))
    gpt_response = llm(history_langchain_format)
    return gpt_response.content

gr.ChatInterface(predict).launch()

### Usando LangChain com OpenAI com streaming

In [ ]:
from google.colab import userdata
from langchain.chat_models import ChatOpenAI
from langchain.schema import AIMessage, HumanMessage
import openai
import gradio as gr
import os

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_KEY')

llm = ChatOpenAI(temperature=1.0, model='gpt-4o', streaming=True)

def predict(message, history):
    history_langchain_format = []
    for human, ai in history:
        history_langchain_format.append(HumanMessage(content=human))
        history_langchain_format.append(AIMessage(content=ai))
    history_langchain_format.append(HumanMessage(content=message))

    # Initialize the response generator
    response_generator = llm.stream(history_langchain_format)

    # Stream responses
    response_content = ""
    for response in response_generator:
        response_content += response.content
        yield response_content

gr.ChatInterface(predict).launch()


### Usando LangChain agora com Gemini

In [1]:
!pip install -q langchain_google_genai

In [ ]:
from google.colab import userdata
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.schema import AIMessage, HumanMessage
import gradio as gr
import os

# Set the environment variable for Gemini API key
os.environ["GOOGLE_API_KEY"] = userdata.get('gemini_api')

# Initialize the Gemini LLM with streaming support
llm = ChatGoogleGenerativeAI(temperature=1.0, model='gemini-pro', streaming=True)  # Adjust the model name as needed

def predict(message, history):
    history_langchain_format = []
    for human, ai in history:
        history_langchain_format.append(HumanMessage(content=human))
        history_langchain_format.append(AIMessage(content=ai))
    history_langchain_format.append(HumanMessage(content=message))

    # Initialize the response generator for streaming
    response_generator = llm.stream(history_langchain_format)

    # Stream responses
    response_content = ""
    for response in response_generator:
        response_content += response.content
        yield response_content

# Launch the Gradio chat interface with the predict function
gr.ChatInterface(predict).launch()
